# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse   Shipments           Customer  \
0  Jan 31 2023  8:30AM  256224        10  0086340314  ISDIN Corporation   
1  Jan 31 2023  8:30AM  256224        10  0086340317  ISDIN Corporation   
2  Jan 31 2023  8:30AM  256224        10  0086340318  ISDIN Corporation   
3  Jan 31 2023  8:30AM  256224        10  0086340315  ISDIN Corporation   
4  Jan 31 2023  8:30AM  256224        10  0086340316  ISDIN Corporation   
5  Jan 31 2023  8:30AM  256248        10  0086340323  ISDIN Corporation   
6  Jan 31 2023  8:30AM  256224        10  0086340340  ISDIN Corporation   
7  Jan 31 2023  8:30AM  256224        10  0086340338  ISDIN Corporation   
8  Jan 31 2023  8:30AM  256248        10  0086340336  ISDIN Corporation   
9  Jan 31 2023  8:30AM  256224        10  0086340341  ISDIN Corporation   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
21  256246       Released          1
22  256247      Completed          1
23  256247       Released          3
24  256248       Released          8
25  256249       Released         13

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
256245                NaN        NaN       1.0
256246                NaN        NaN       1.0
256247                1.0        NaN       3.0
256248                NaN        NaN       8.0
256249                NaN        NaN      13.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
256149                0.0        0.0       2.0
256172                0.0        0.0       1.0
256176                0.0        0.0       1.0
256178                0.0        1.0       0.0
256186               47.0        5.0       3.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
256149                  0          0         2
256172                  0          0         1
256176                  0          0         1
256178                  0          1         0
256186                 47          5         3

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               256149          0          0         2
1               256172          0          0         1
2               256176          0          0         1
3               256178          0          1         0
4               256186         47          5         3

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               256149                            2
1               256172                            1
2               256176                            1
3               256178                   1         
4               256186        47         5        3

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                      Customer
0   Jan 31 2023  8:30AM  256224        10             ISDIN Corporation
5   Jan 31 2023  8:30AM  256248        10             ISDIN Corporation
24  Jan 31 2023  8:30AM  256249        10             ISDIN Corporation
37  Jan 31 2023  8:30AM  256247        10             ISDIN Corporation
43  Jan 31 2023  8:30AM  256230        10  Snap Medical Industries, LLC
55  Jan 31 2023  8:30AM  256246        10                Eye Pharma Inc
56  Jan 30 2023  4:04PM  256245        22             NBTY Global, Inc.
57  Jan 30 2023  3:45PM  256236        19           AdvaGen Pharma, Ltd
86  Jan 30 2023  3:39PM  256234        22             NBTY Global, Inc.
87  Jan 30 2023  2:58PM  256227        22             NBTY Global, Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                      Customer  \
0  Jan 31 2023  8:30AM  256224        10             ISDIN Corporation   
1  Jan 31 2023  8:30AM  256248        10             ISDIN Corporation   
2  Jan 31 2023  8:30AM  256249        10             ISDIN Corporation   
3  Jan 31 2023  8:30AM  256247        10             ISDIN Corporation   
4  Jan 31 2023  8:30AM  256230        10  Snap Medical Industries, LLC   
5  Jan 31 2023  8:30AM  256246        10                Eye Pharma Inc   
6  Jan 30 2023  4:04PM  256245        22             NBTY Global, Inc.   
7  Jan 30 2023  3:45PM  256236        19           AdvaGen Pharma, Ltd   
8  Jan 30 2023  3:39PM  256234        22             NBTY Global, Inc.   
9  Jan 30 2023  2:58PM  256227        22             NBTY Global, Inc.   

  Completed Executing Released  
0                           19  
1                            8  
2                           13  
3         1                  3  
4                           11  
5                            1  
6                            1  
7                           29  
8                            1  
9                            1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                      Customer  \
0  Jan 31 2023  8:30AM  256224        10             ISDIN Corporation   
1  Jan 31 2023  8:30AM  256248        10             ISDIN Corporation   
2  Jan 31 2023  8:30AM  256249        10             ISDIN Corporation   
3  Jan 31 2023  8:30AM  256247        10             ISDIN Corporation   
4  Jan 31 2023  8:30AM  256230        10  Snap Medical Industries, LLC   
5  Jan 31 2023  8:30AM  256246        10                Eye Pharma Inc   
6  Jan 30 2023  4:04PM  256245        22             NBTY Global, Inc.   
7  Jan 30 2023  3:45PM  256236        19           AdvaGen Pharma, Ltd   
8  Jan 30 2023  3:39PM  256234        22             NBTY Global, Inc.   
9  Jan 30 2023  2:58PM  256227        22             NBTY Global, Inc.   

  Released Executing Completed  
0       19                      
1        8                      
2       13                      
3        3                   1  
4       11                      
5        1                      
6        1                      
7       29                      
8        1                      
9        1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                      Customer  \
0  Jan 31 2023  8:30AM  256224        10             ISDIN Corporation   
1  Jan 31 2023  8:30AM  256248        10             ISDIN Corporation   
2  Jan 31 2023  8:30AM  256249        10             ISDIN Corporation   
3  Jan 31 2023  8:30AM  256247        10             ISDIN Corporation   
4  Jan 31 2023  8:30AM  256230        10  Snap Medical Industries, LLC   

  Released Executing Completed  
0       19                      
1        8                      
2       13                      
3        3                   1  
4       11

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                      Customer  \
0  Jan 31 2023  8:30AM  256224        10             ISDIN Corporation   
1  Jan 31 2023  8:30AM  256248        10             ISDIN Corporation   
2  Jan 31 2023  8:30AM  256249        10             ISDIN Corporation   
3  Jan 31 2023  8:30AM  256247        10             ISDIN Corporation   
4  Jan 31 2023  8:30AM  256230        10  Snap Medical Industries, LLC   

   Released  Executing  Completed  
0      19.0        NaN        NaN  
1       8.0        NaN        NaN  
2      13.0        NaN        NaN  
3       3.0        NaN        1.0  
4      11.0        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                      Customer  \
0  Jan 31 2023  8:30AM  256224        10             ISDIN Corporation   
1  Jan 31 2023  8:30AM  256248        10             ISDIN Corporation   
2  Jan 31 2023  8:30AM  256249        10             ISDIN Corporation   
3  Jan 31 2023  8:30AM  256247        10             ISDIN Corporation   
4  Jan 31 2023  8:30AM  256230        10  Snap Medical Industries, LLC   

   Released  Executing  Completed  
0      19.0        0.0        0.0  
1       8.0        0.0        0.0  
2      13.0        0.0        0.0  
3       3.0        0.0        1.0  
4      11.0        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         3074584     117.0        0.0        1.0
12          512374       1.0        1.0        0.0
19          512422      32.0        5.0       47.0
21          512390       2.0        0.0        0.0
22         1281157       5.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  3074584     117.0        0.0        1.0
1        12   512374       1.0        1.0        0.0
2        19   512422      32.0        5.0       47.0
3        21   512390       2.0        0.0        0.0
4        22  1281157       5.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10     117.0        0.0        1.0
1        12       1.0        1.0        0.0
2        19      32.0        5.0       47.0
3        21       2.0        0.0        0.0
4        22       5.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released  117.0
1        12  Released    1.0
2        19  Released   32.0
3        21  Released    2.0
4        22  Released    5.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10   12    19   21   22
Status                               
Completed    1.0  0.0  47.0  0.0  0.0
Executing    0.0  1.0   5.0  0.0  0.0
Released   117.0  1.0  32.0  2.0  5.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10   12    19   21   22
0          Completed    1.0  0.0  47.0  0.0  0.0
1          Executing    0.0  1.0   5.0  0.0  0.0
2           Released  117.0  1.0  32.0  2.0  5.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10   12    19   21   22
0  Completed    1.0  0.0  47.0  0.0  0.0
1  Executing    0.0  1.0   5.0  0.0  0.0
2   Released  117.0  1.0  32.0  2.0  5.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()